In [3]:
import requests
from bs4 import BeautifulSoup
import time

In [5]:
url = "https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [31]:
valid_links = ["/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"]
for i in range(10):
    valid_links = list(set(valid_links))
    baselink = "https://pt.wikipedia.org"
    url = f"{baselink}{valid_links[i]}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    todos_links = soup.find(id="bodyContent").find_all("a")
    for link in todos_links:
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if link["href"].find(":") == -1:
                valid_links.append(link["href"])
    print(valid_links[i])
    time.sleep(1)


/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal
/wiki/Batalha_de_Cartum_(2023-2025)
/wiki/RFC
/wiki/2012
/wiki/%C3%8Dndia
/wiki/Nova_Jersey
/wiki/Alligator_mississippiensis
/wiki/Alligator_mississippiensis
/wiki/Energia_e%C3%B3lica
/wiki/Nova_Jersey


In [32]:
print(valid_links)

['/wiki/Imp%C3%A9rio_Portugu%C3%AAs', '/wiki/NASA', '/wiki/Calend%C3%A1rio_r%C3%BAnico', '/wiki/D%C3%A9cada_de_1830', '/wiki/Jogos_da_Commonwealth_de_2010', '/wiki/Territ%C3%B3rio', '/wiki/National_Center_for_Biotechnology_Information', '/wiki/Alligator_mississippiensis', '/wiki/Energia_e%C3%B3lica', '/wiki/Nova_Jersey', '/wiki/Egito', '/wiki/Jacar%C3%A9-de-papo-amarelo', '/wiki/PubMed_Identifier', '/wiki/Vela_(n%C3%A1utica)', '/wiki/Taxila', '/wiki/Gases_de_efeito_de_estufa', '/wiki/Neol%C3%ADtico', '/wiki/Editora_Abril', '/wiki/Chaturanga', '/wiki/Biblioteca_Nacional_da_Fran%C3%A7a', '/wiki/Romaniza%C3%A7%C3%A3o_(lingu%C3%ADstica)', '/wiki/Poder_executivo', '/wiki/Constru%C3%A7%C3%A3o', '/wiki/Escorpi%C3%A3o_(astrologia)', '/wiki/Quarta_c%C3%BApula_do_BRICS', '/wiki/Velho_Continente', '/wiki/Draupadi_Murmu', '/wiki/Energia_hidroel%C3%A9trica', '/wiki/Companhia_Brit%C3%A2nica_das_%C3%8Dndias_Orientais', '/wiki/Ver%C3%A3o', '/wiki/Rep%C3%BAblica_parlamentarista', '/wiki/Bangalore', '/w

In [49]:
import requests
from bs4 import BeautifulSoup

def coletar_links_wikipedia(inicio_url, limite=100):
    fila = [inicio_url]
    visitados = set()
    links_coletados = [] 
    
    while len(links_coletados) < limite:
        # Pega o último link da fila e guarda para o acesso
        url_atual = fila.pop()
        
        # Se o link já foi visitado, pula
        if url_atual in visitados:
            continue 
        
        # Se for um link novo, guarda nos visitados e coleta o link
        visitados.add(url_atual)
        links_coletados.append(url_atual)

        # Carrega a página
        response = requests.get(url_atual)
        soup = BeautifulSoup(response.text, "html.parser")

        # Procura por todos os links na página carregada e itera sob eles
        for link in soup.find_all("a", href=True):
            
            # Verifica se é um verbete
            verbete = link["href"]
            if verbete.startswith("/wiki/") and link["href"].startswith("/wiki/") and link["href"].find(":") == -1:
                
                # Verifica se algum dos links já foi guardado
                novo_link = "https://pt.wikipedia.org" + verbete
                if novo_link not in visitados:
                    # Guarda o link na fila (o último link guardado da página será o próximo a ser carregado)
                    fila.append(novo_link)

        print(links_coletados[-1])
        # print(len(links_coletados))
        time.sleep(0.5)
    return links_coletados


url_inicial = "https://pt.wikipedia.org/wiki/Wikipédia:Página_principal"
links = coletar_links_wikipedia(url_inicial)

print(links[:10])


https://pt.wikipedia.org/wiki/Wikipédia:Página_principal
https://pt.wikipedia.org/wiki/Licen%C3%A7a_livre
https://pt.wikipedia.org/wiki/Encyclop%C3%A6dia_Britannica
https://pt.wikipedia.org/wiki/Biblioteca_Nacional_de_Israel
https://pt.wikipedia.org/wiki/Sistema_Universit%C3%A1rio_de_Documenta%C3%A7%C3%A3o
https://pt.wikipedia.org/wiki/Online_Computer_Library_Center
https://pt.wikipedia.org/wiki/Biblioteca_Nacional_da_Pol%C3%B3nia
https://pt.wikipedia.org/wiki/Registo_da_Mem%C3%B3ria_do_Mundo
https://pt.wikipedia.org/wiki/Lista_do_Patrim%C3%B4nio_Mundial_em_Perigo
https://pt.wikipedia.org/wiki/Organiza%C3%A7%C3%A3o_das_Na%C3%A7%C3%B5es_Unidas_para_a_Educa%C3%A7%C3%A3o,_a_Ci%C3%AAncia_e_a_Cultura
https://pt.wikipedia.org/wiki/Sistema_Unificado_de_Linguagem_M%C3%A9dica
https://pt.wikipedia.org/wiki/Medical_Subject_Headings
https://pt.wikipedia.org/wiki/N%C3%BAmero_de_controle_da_Biblioteca_do_Congresso


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def coletar_links(url, limite=5000):
    # Fila contém uma série de links (todos os links das páginas acessadas). Set contém os links únicos já visitados.
    fila = [url]
    visitados = set()

    # Se menos de 5k links foram guardados, conitnua execução
    while fila and len(visitados) < limite:
        try:
            # Acessa o último link inserido na fila
            atual = fila.pop()

            # Verifica se esse link já foi acessado. Se não, continua a execução e adiciona a lista de visitados.
            if atual in visitados: 
                continue
            visitados.add(atual)

            # Carrega a página atual.
            r = requests.get(atual)
            s = BeautifulSoup(r.text, 'html.parser')

            titulo = s.select(".mw-page-title-main")
            nome_arquivo = titulo[0].text

            with open(f"pages/{nome_arquivo}.html", "w", encoding="utf-8") as f:
                conteudo = r.content.decode("utf-8")
                f.write(conteudo)

            # Itera sobre todos os links da página atual.
            for link in s.find_all('a', href=True):

                # Verifica se a página é um verbete.
                pagina = link['href']
                if pagina.startswith('/wiki/') and ':' not in pagina:

                    # Se algum dos links da página carregada já tiver sido visitada, pula.
                    link_atual = 'https://pt.wikipedia.org' + pagina
                    if link_atual in visitados:
                        continue
                    # Adiciona todos que forem verbetes à fila. O último link coletado será a próxima página a ser aberta.
                    fila.append('https://pt.wikipedia.org' + pagina)

            print(len(visitados), end="\r")
            time.sleep(0.7)

        # Evita páginas sem título (titulo[0].text)
        except IndexError:
            continue

        # Evita erros de mal formatação de título
        except FileNotFoundError:
            continue

    return list(visitados)

links = coletar_links('https://pt.wikipedia.org/wiki/Wikipédia:Página_principal')


In [43]:
print(len(links))

10


In [44]:
for link in links:
    print(link)

https://pt.wikipedia.org/wiki/Wikipédia:Página_principal
https://pt.wikipedia.org/wiki/Licen%C3%A7a_livre
https://pt.wikipedia.org/wiki/Encyclop%C3%A6dia_Britannica
https://pt.wikipedia.org/wiki/Biblioteca_Nacional_de_Israel
https://pt.wikipedia.org/wiki/Sistema_Universit%C3%A1rio_de_Documenta%C3%A7%C3%A3o
https://pt.wikipedia.org/wiki/Online_Computer_Library_Center
https://pt.wikipedia.org/wiki/Biblioteca_Nacional_da_Pol%C3%B3nia
https://pt.wikipedia.org/wiki/Registo_da_Mem%C3%B3ria_do_Mundo
https://pt.wikipedia.org/wiki/Lista_do_Patrim%C3%B4nio_Mundial_em_Perigo
https://pt.wikipedia.org/wiki/Organiza%C3%A7%C3%A3o_das_Na%C3%A7%C3%B5es_Unidas_para_a_Educa%C3%A7%C3%A3o,_a_Ci%C3%AAncia_e_a_Cultura
